In [47]:
# Import all required libraries
from sklearn.cluster import KMeans
import nltk
import numpy as np
import re
import pandas as pd 
import pylab as pl
import matplotlib.pyplot as plt

from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
#from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import pyplot

#plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'



import datashader as ds
import datashader.transfer_functions as tf

In [48]:
df=pd.read_csv("train.csv",engine='python')

In [49]:
df.head()

,ItemID,SentimentText
0,1,is so sad for my APL frie...
1,2,I missed the New Moon trail...
2,3,omg its already 7:30 :O
3,4,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,i think mi bf is cheating on me!!! ...


In [50]:
df=df.rename({'SentimentText': 'full_text'},axis=1)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99989 entries, 0 to 99988
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ItemID     99989 non-null  int64 
 1   full_text  99989 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [52]:
df.drop(['ItemID'],axis=1,inplace = True)

In [53]:
df.head()

,full_text
0,is so sad for my APL frie...
1,I missed the New Moon trail...
2,omg its already 7:30 :O
3,.. Omgaga. Im sooo im gunna CRy. I'...
4,i think mi bf is cheating on me!!! ...


In [54]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [55]:
df['Clean_text'] = np.vectorize(remove_pattern)(df['full_text'], "@[\w]*")

In [56]:
# remove special characters, numbers, punctuations
df['Clean_text'] = df['Clean_text'].str.replace("[^a-zA-Z#]", " ")

In [57]:
df.head(5)

,full_text,Clean_text
0,is so sad for my APL frie...,is so sad for my APL frie...
1,I missed the New Moon trail...,I missed the New Moon trail...
2,omg its already 7:30 :O,omg its already O
3,.. Omgaga. Im sooo im gunna CRy. I'...,Omgaga Im sooo im gunna CRy I ...
4,i think mi bf is cheating on me!!! ...,i think mi bf is cheating on me ...


In [58]:
tokenized_tweet = df['Clean_text'].apply(lambda x: x.split())

In [59]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0                  [is, so, sad, for, my, apl, friend]
1                   [I, miss, the, new, moon, trailer]
2                                [omg, it, alreadi, O]
3    [omgaga, Im, sooo, im, gunna, cri, I, ve, been...
4          [i, think, mi, bf, is, cheat, on, me, T, T]
Name: Clean_text, dtype: object

In [60]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

df['Clean_text'] = tokenized_tweet

In [61]:
df.loc[:,('full_text','Clean_text')]

,full_text,Clean_text
0,is so sad for my APL frie...,is so sad for my apl friend
1,I missed the New Moon trail...,I miss the new moon trailer
2,omg its already 7:30 :O,omg it alreadi O
3,.. Omgaga. Im sooo im gunna CRy. I'...,omgaga Im sooo im gunna cri I ve been at thi d...
4,i think mi bf is cheating on me!!! ...,i think mi bf is cheat on me T T
...,...,...
99984,@Cupcake seems like a repeating problem hop...,seem like a repeat problem hope you re abl to ...
99985,@cupcake__ arrrr we both replied to each other...,arrrr we both repli to each other over differ ...
99986,@CuPcAkE_2120 ya i thought so,ya i thought so
99987,@Cupcake_Dollie Yes. Yes. I'm glad you had mor...,ye ye I m glad you had more fun with me


In [62]:
#tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# Considering 3 grams and mimnimum frq as 0
tf_idf_vect = CountVectorizer(analyzer='word',ngram_range=(1,1),stop_words='english', min_df = 0.0001)
tf_idf_vect.fit(df['Clean_text'])
desc_matrix = tf_idf_vect.transform(df["Clean_text"])

In [ ]:
# implement kmeans
num_clusters = 2
km = KMeans(n_clusters=num_clusters)
km.fit(desc_matrix)
clusters = km.labels_.tolist()

In [ ]:
# create DataFrame films from all of the input files.
tweets = {'Tweet': df["Clean_text"].tolist(), 'Cluster': clusters}
frame = pd.DataFrame(tweets, index = [clusters])
frame.tail(20)

In [ ]:
frame['Cluster'].value_counts()

In [ ]:
frame[frame['Cluster'] == 1].head()

In [ ]:
frame[frame['Cluster'] == 0].head()

# ---------------------------------------------------------------------------------------